## 아고다 전처리
- re, hanspell, konlpy, gensim
- 한글만 뽑기, 중복 제거
- 특수문자 제거
- 의미 없는 자음, 모음 제거
- 이모티콘 제거
- 맞춤법 고치기 (띄어쓰기, 맞춤법 등)

### 파일 import 

In [1]:
import numpy as np 
import tensorflow as tf 
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
agoda1 = pd.read_csv('data/agoda_review_4.csv')
agoda2 = pd.read_csv('data/agoda_review_10.csv')
agoda3 = pd.read_csv('data/agoda_review_12.csv')

In [5]:
agoda1.head(1)

,호텔 이름,리뷰
0,"조선 팰리스 서울 강남 럭셔리 컬렉션 호텔 (Josun Palace, a Luxur...",대만족스러운 호텔입니다! 호캉스홀릭 카페 추천으로 다녀왔는데 넘 좋아써요


In [6]:
agoda2.head(1)

,호텔 이름,리뷰
0,나인트리 프리미어 호텔 명동 2 (Nine Tree Premier Hotel Mye...,해외거주중 아이 방학에 맞춰 한국에 갔습니다. 한달 가량 머물 숙소를 찾다가 나인트...


In [7]:
agoda3.head(1)

,호텔 이름,리뷰
0,머큐어 앰배서더 서울 강남 쏘도베 (Mercure Ambassador Seoul G...,"침대 시트에 검은 자국이 좀 있었지만 그래도 위치,가격, 룸 분위기 , 직원 태도,..."


### 데이터 전처리

In [8]:
# 데이터 합치기
agoda1.columns = ['hotel', 'review']
agoda2.columns = ['hotel', 'review']
agoda3.columns = ['hotel', 'review']

In [9]:
agoda = pd.concat([agoda1, agoda2, agoda3])
agoda.head(3)

,hotel,review
0,"조선 팰리스 서울 강남 럭셔리 컬렉션 호텔 (Josun Palace, a Luxur...",대만족스러운 호텔입니다! 호캉스홀릭 카페 추천으로 다녀왔는데 넘 좋아써요
1,신라스테이 마포 (Shilla Stay Mapo),모난곳 없는 좋은 숙소!
2,신라스테이 마포 (Shilla Stay Mapo),아고다를 통해 예약을 했습니다. 아주 저렴하게 이용할 수 있었습니다. 모든게 만족스...


In [10]:
agoda.info

<bound method DataFrame.info of                                                   hotel  \
0     조선 팰리스 서울 강남 럭셔리 컬렉션 호텔 (Josun Palace, a Luxur...   
1                           신라스테이 마포 (Shilla Stay Mapo)   
2                           신라스테이 마포 (Shilla Stay Mapo)   
3                           신라스테이 마포 (Shilla Stay Mapo)   
4                           신라스테이 마포 (Shilla Stay Mapo)   
...                                                 ...   
1387  머큐어 앰배서더 서울 강남 쏘도베 (Mercure Ambassador Seoul G...   
1388  머큐어 앰배서더 서울 강남 쏘도베 (Mercure Ambassador Seoul G...   
1389  머큐어 앰배서더 서울 강남 쏘도베 (Mercure Ambassador Seoul G...   
1390  머큐어 앰배서더 서울 강남 쏘도베 (Mercure Ambassador Seoul G...   
1391  머큐어 앰배서더 서울 강남 쏘도베 (Mercure Ambassador Seoul G...   

                                                 review  
0              대만족스러운 호텔입니다! 호캉스홀릭 카페 추천으로 다녀왔는데 넘 좋아써요  
1                                         모난곳 없는 좋은 숙소!  
2     아고다를 통해 예약을 했습니다. 아주 저렴하게 이용할 수 있었습니다. 모든게 만족스...  
3     옷장 위 쪽엔 먼지와 누군가의 양말이 

In [11]:
# 결측치 확인
agoda.isnull().sum()

hotel     0
review    7
dtype: int64

In [12]:
# 결측치 삭제
agoda = agoda.dropna(axis=0)

In [13]:
len(agoda)

9770

In [14]:
agoda.isnull().sum()

hotel     0
review    0
dtype: int64

In [15]:
# 데이터 중복 여부 확인
agoda.review.nunique()

9556

In [16]:
# 중복 데이터 제거
agoda = agoda.drop_duplicates('review', keep='first')

In [17]:
len(agoda.review.values)

9556

In [18]:
agoda.tail(5)

,hotel,review
1386,머큐어 앰배서더 서울 강남 쏘도베 (Mercure Ambassador Seoul G...,整體感覺不錯。房間仍然不大，但比明洞大的多。出巷口就是地鐵站和機場大巴。比較沒想到酒店竟沒有...
1387,머큐어 앰배서더 서울 강남 쏘도베 (Mercure Ambassador Seoul G...,離江南站一個站的地方，雖然沒有很熱鬧，但交通便利，距離地鐵站走路只要3分鐘。附近也有許多餐廳...
1388,머큐어 앰배서더 서울 강남 쏘도베 (Mercure Ambassador Seoul G...,出差的關係住在江南區 仁川機場可以直接撘乘6020 bus (KWR ) 直接到到驛三地鐵站...
1389,머큐어 앰배서더 서울 강남 쏘도베 (Mercure Ambassador Seoul G...,"近地鐵,到機場交通方便."
1391,머큐어 앰배서더 서울 강남 쏘도베 (Mercure Ambassador Seoul G...,飯店人員服務佳，位置優越，離地鐵很近，且附近餐廳不少，對面有便利商店，但飯店的居住環境及舒適...


### 텍스트 전처리

In [19]:
# 한글과 공백 이외는 제거
agoda['review'] = agoda.review.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '')
agoda.tail(5)

<ipython-input-19-ee439f4431e2>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  agoda['review'] = agoda.review.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '')


,hotel,review
1386,머큐어 앰배서더 서울 강남 쏘도베 (Mercure Ambassador Seoul G...,
1387,머큐어 앰배서더 서울 강남 쏘도베 (Mercure Ambassador Seoul G...,
1388,머큐어 앰배서더 서울 강남 쏘도베 (Mercure Ambassador Seoul G...,
1389,머큐어 앰배서더 서울 강남 쏘도베 (Mercure Ambassador Seoul G...,
1391,머큐어 앰배서더 서울 강남 쏘도베 (Mercure Ambassador Seoul G...,


In [20]:
agoda['review'].replace('', np.nan, inplace=True)
agoda.review.isnull().sum()

943

In [21]:
agoda = agoda.dropna(how='any')
agoda.shape

(8613, 2)

In [22]:
agoda.shape()

(8613, 2)

In [29]:
# 공백만 있는 행 제거
agoda['review'] = agoda.review.str.replace(' ', '')

In [31]:
agoda['review'].replace('', np.nan, inplace=True)
agoda.review.isnull().sum()

2939

In [32]:
agoda = agoda.dropna(how='any')
agoda.shape

(5674, 3)

In [33]:
# 문장길이 컬럼 추가
agoda['length'] = agoda['review'].apply(lambda x: len(str(x)))
agoda['review'] = agoda['review'].astype('str')
agoda

<ipython-input-33-535affc35b55>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agoda['length'] = agoda['review'].apply(lambda x: len(str(x)))
<ipython-input-33-535affc35b55>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agoda['review'] = agoda['review'].astype('str')


,hotel,review,length
0,"조선 팰리스 서울 강남 럭셔리 컬렉션 호텔 (Josun Palace, a Luxur...",대만족스러운호텔입니다호캉스홀릭카페추천으로다녀왔는데넘좋아써요,32
1,신라스테이 마포 (Shilla Stay Mapo),모난곳없는좋은숙소,9
2,신라스테이 마포 (Shilla Stay Mapo),아고다를통해예약을했습니다아주저렴하게이용할수있었습니다모든게만족스러웠죠친절과청결은최상이...,69
3,신라스테이 마포 (Shilla Stay Mapo),옷장위쪽엔먼지와누군가의양말이그대로있었어요쇼파엔다른투숙객이만든얼룩이그대로있구요,42
4,신라스테이 마포 (Shilla Stay Mapo),신라스테이가격도좋고편안함도좋고푹쉬다가기좋아요,24
...,...,...,...
327,머큐어 앰배서더 서울 강남 쏘도베 (Mercure Ambassador Seoul G...,그냥일반인이라면특히연세가있으신분께는비추입니다방과욕실모두협소합니다그냥잠만잘수있는정도이...,84
328,머큐어 앰배서더 서울 강남 쏘도베 (Mercure Ambassador Seoul G...,아쉬웠던점은특히모닝커피를룸에서도마실수있도록했으면시이후에룸서비스가되니커피도팟으로서비스...,56
329,머큐어 앰배서더 서울 강남 쏘도베 (Mercure Ambassador Seoul G...,새로지어깨끗함화장실이너무좁음여는데문제가있음나머지는적당함,30
574,머큐어 앰배서더 서울 강남 쏘도베 (Mercure Ambassador Seoul G...,역삼역,3


In [37]:
# 문장 길이순으로 정렬, 인덱스 리셋
agoda = agoda.sort_values(by=['length'], axis=0)
agoda = agoda.reset_index(drop=True)

In [39]:
agoda.to_csv('아고다_띄어쓰기없음_글자개수순정렬.csv', index=False)

In [38]:
agoda.

hotel     루체브릿지 호텔 (Luce Bridge Hotel)
review                               ㅁ
length                               1
Name: 0, dtype: object